In [1]:
from td_exercise_environment import Environment
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()

puzzle_width = 8
puzzle_height = 8
no_mouses = 18

# This should be in put
start_x = 0
start_y = 0
env = Environment(puzzle_height, puzzle_width)
env.set_start_state(start_x, start_y)

Loading BokehJS ...

In [10]:
import numpy as np
mouses = []

# for i in range(no_mouses):
#     mouses.append((np.random.randint(0, puzzle_width), np.random.randint(0, puzzle_height)))

mouses.append((0, 2))
mouses.append((1, 1))
mouses.append((1, 3))
mouses.append((2, 1))
mouses.append((3, 1))
mouses.append((3, 3))
mouses.append((4, 3))
mouses.append((4, 4))
mouses.append((4, 5))
mouses.append((4, 6))
mouses.append((5, 1))
mouses.append((5, 2))
mouses.append((5, 3))
mouses.append((6, 1))
mouses.append((6, 6))
mouses.append((6, 7))
mouses.append((7, 1))
env.set_mouses(mouses)

In [11]:
from bokeh.models.glyphs import Rect

rect_lst = []
WIDTH=1
HEIGHT=1

def draw_rectangles(p, env):
    global rect_lst, WIDTH, HEIGHT
    for i in range(env._size_y):
        row = []
        for j in range(env._size_x):
            rect = Rect(x=j, y=-i, width=WIDTH, height=HEIGHT, 
                        fill_color="#CAB2D6", line_width=1.0, line_color = "#000000")
            p.add_glyph(rect)
            row.append(rect)
        rect_lst.append(row)
    for mouse in env.mouses:
        rect_lst[mouse[1]][mouse[0]].fill_color="#ff0000"
                        
def update_rects(Q, env):
    previous_states = []
    state = env.reset()
    done = False
    while not done:
        if state in previous_states:
            print("mouse is going in circles")
            break
        previous_states.append(state)
        rect_lst[state._y][state._x].fill_color="#69ea69"
        best_next_action = max(a.Q[state].items(), key=operator.itemgetter(1))[0]
        next_state, _, done = env.step(best_next_action)
        state = next_state
    
p = figure()
draw_rectangles(p, env)
p.axis.visible = False

show(p)

In [12]:
from td_exercise_agent import get_random_action, make_epsilon_greedy_policy, Agent
from random import random
import operator

# THIS SHOULD BE PUT IN
num_episodes=500
epsilon=0.2
discount_factor=0.8
alpha=0.5

a = Agent(env)
a.init_q_values()

policy = make_epsilon_greedy_policy(a.Q, epsilon)

for i in range(num_episodes):

    # Initialize state
    state = env.reset()

    while True:
        # Choose A from S using policy derived from Q
        action_probs = policy(state)
        action = get_random_action(action_probs)

        # Do a step in the environment
        next_state, reward, done = env.step(action)

        # TD control
        best_next_action = max(a.Q[next_state].items(), key=operator.itemgetter(1))[0]
        td_target = reward + discount_factor * a.Q[next_state][best_next_action]
        td_delta = td_target - a.Q[state][action]
        a.Q[state][action] += alpha * td_delta

        # Check if the current state is terminal
        if done:
            break

        state = next_state
        
# Update the view with the new value for a.Q[state][action]
# update_labels(a.Q)
update_rects(a.Q, env)

show(p)